In [2]:
import pandas as pd
import numpy as np
from numpy import mean, std
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import classification_report
import itertools
from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import cohen_kappa_score, make_scorer
kappa_scorer = make_scorer(cohen_kappa_score)

In [3]:
# Load data from Url
url = 'https://drive.google.com/uc?id=1aCyjeOGsLe2Tti96vg3xPl1kRhzpJ1wl'
df = pd.read_csv(url)
# One-hot encode the 'climate_class' categorical feature
df_encoded = pd.get_dummies(df, columns=['climate_class'], prefix=['climate_class'], drop_first=False)

In [4]:
df_encoded

,final_duration,final_cost,adjustment_cost,final_change_cost,delay_class,climate_class_C,climate_class_HD,climate_class_MR,climate_class_SA,climate_class_SMR,climate_class_VHD,climate_class_VHH
0,1560,5.080000e+10,2.230000e+10,5.082216e+10,0,False,False,False,False,False,True,False
1,1297,6.010000e+09,0.000000e+00,6.007480e+09,0,False,False,False,False,False,True,False
2,933,1.946772e+10,1.215343e+10,0.000000e+00,0,False,False,True,False,False,False,False
3,1313,4.749459e+10,2.122647e+10,4.749459e+10,0,True,False,False,False,False,False,False
4,749,1.282204e+10,0.000000e+00,1.280483e+10,0,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
434,1453,3.274304e+10,1.615689e+10,0.000000e+00,1,False,False,True,False,False,False,False
435,1165,3.224325e+10,3.308631e+09,0.000000e+00,1,True,False,False,False,False,False,False
436,1637,2.314913e+10,1.382445e+10,0.000000e+00,1,False,False,False,False,False,True,False
437,585,2.652207e+09,0.000000e+00,2.623708e+09,1,False,False,False,False,False,True,False


In [5]:
filtered_df = df_encoded

In [6]:
# Separate features and target
X = filtered_df[['final_duration', 'final_cost', 'adjustment_cost', 'final_change_cost','climate_class_C', 'climate_class_HD', 'climate_class_MR', 'climate_class_SA', 'climate_class_SMR', 'climate_class_VHD', 'climate_class_VHH']]
y = filtered_df['delay_class']

# Normalize numerical features
scaler = StandardScaler()
X[['final_duration', 'final_cost', 'adjustment_cost', 'final_change_cost']] = scaler.fit_transform(X[['final_duration', 'final_cost', 'adjustment_cost', 'final_change_cost']])

C:\Users\Sep\AppData\Local\Temp\ipykernel_6488\1544043725.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['final_duration', 'final_cost', 'adjustment_cost', 'final_change_cost']] = scaler.fit_transform(X[['final_duration', 'final_cost', 'adjustment_cost', 'final_change_cost']])


In [7]:
# Show data sample
filtered_df

,final_duration,final_cost,adjustment_cost,final_change_cost,delay_class,climate_class_C,climate_class_HD,climate_class_MR,climate_class_SA,climate_class_SMR,climate_class_VHD,climate_class_VHH
0,1560,5.080000e+10,2.230000e+10,5.082216e+10,0,False,False,False,False,False,True,False
1,1297,6.010000e+09,0.000000e+00,6.007480e+09,0,False,False,False,False,False,True,False
2,933,1.946772e+10,1.215343e+10,0.000000e+00,0,False,False,True,False,False,False,False
3,1313,4.749459e+10,2.122647e+10,4.749459e+10,0,True,False,False,False,False,False,False
4,749,1.282204e+10,0.000000e+00,1.280483e+10,0,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
434,1453,3.274304e+10,1.615689e+10,0.000000e+00,1,False,False,True,False,False,False,False
435,1165,3.224325e+10,3.308631e+09,0.000000e+00,1,True,False,False,False,False,False,False
436,1637,2.314913e+10,1.382445e+10,0.000000e+00,1,False,False,False,False,False,True,False
437,585,2.652207e+09,0.000000e+00,2.623708e+09,1,False,False,False,False,False,True,False


In [8]:
X.shape

(439, 11)

In [9]:
# Apply outlier detection
outlier_detector = EllipticEnvelope()
outliers = outlier_detector.fit_predict(X[['final_duration', 'final_cost', 'adjustment_cost', 'final_change_cost', 'climate_class_C', 'climate_class_HD', 'climate_class_MR', 'climate_class_SA', 'climate_class_SMR', 'climate_class_VHD', 'climate_class_VHH']])
X = X[outliers == 1]
y = y[outliers == 1]

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [11]:
X.shape

(395, 11)

# SVC

In [12]:
from sklearn.svm import SVC
model = SVC()
kernel = ['linear','poly', 'rbf', 'sigmoid']
C = [ 1.0, 2.0, 3.0, 5.0, 10.0, 0.1, 0.01, 100]
grid = dict(kernel=kernel,C=C)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.723661 using {'C': 100, 'kernel': 'rbf'}


In [13]:
model_svm = SVC(C= 100, kernel='rbf')

model=model_svm
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.7271 (0.0470)
f1_macro: 0.7237 (0.0478)
precision_macro: 0.7352 (0.0480)
recall_macro: 0.7277 (0.0466)
kappa_scorer: 0.5908 (0.0704)


# KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
n_neighbors = range(1, 21)
weights = ['uniform', 'distance']
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
grid = dict(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro', error_score=0)

grid_result = grid_search.fit(X, y)
print("Best F1 Score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best F1 Score: 0.863607 using {'algorithm': 'auto', 'n_neighbors': 1, 'weights': 'uniform'}


In [14]:
model_knn = KNeighborsClassifier(algorithm= 'auto', n_neighbors= 1, weights= 'uniform')
model = model_knn
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.8511 (0.0366)
f1_macro: 0.8410 (0.0423)
precision_macro: 0.8683 (0.0339)
recall_macro: 0.8516 (0.0356)
kappa_scorer: 0.7767 (0.0547)


# RandomForest

In [16]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=1)
n_estimators = [50, 100, 120, 150, 200, 300]
max_features = ['sqrt', 'log2', 0.5]
max_depth = [2,6,8,10,12,14,16]
grid = dict(n_estimators=n_estimators,
            max_features=max_features,max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.900766 using {'max_depth': 14, 'max_features': 'sqrt', 'n_estimators': 50}


In [15]:
model_rf = RandomForestClassifier(random_state=1, max_depth=14 , max_features='sqrt', n_estimators=50)

from sklearn.metrics import accuracy_score
history = model_rf.fit(X_train,y_train)
y_hat = model_rf.predict(X_test)

cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_rf, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_rf, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_rf, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_rf, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.8906 (0.0332)
f1_macro: 0.8872 (0.0356)
precision_macro: 0.8989 (0.0309)
recall_macro: 0.8908 (0.0336)
kappa_scorer: 0.8359 (0.0499)


# DT

In [15]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=1)
max_features = ['sqrt', 'log2', 0.5]
max_depth = [2,6,8,10,12,14,16]
grid = dict(max_features=max_features,max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.872435 using {'max_depth': 14, 'max_features': 0.5}


In [16]:
model_dt = DecisionTreeClassifier(random_state=1, max_depth=16 , max_features=0.5)
from sklearn.metrics import accuracy_score
history = model_dt.fit(X_train,y_train)
y_hat = model_dt.predict(X_test)

cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model_dt, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_dt, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_dt, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_dt, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model_dt, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.8365 (0.0428)
f1_macro: 0.8314 (0.0477)
precision_macro: 0.8418 (0.0425)
recall_macro: 0.8370 (0.0430)
kappa_scorer: 0.7547 (0.0642)


# NB

In [17]:
model =  GaussianNB()
var_smoothing= np.logspace(0,-9, num=5)
grid = dict(var_smoothing=var_smoothing)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.418914 using {'var_smoothing': 0.005623413251903491}


In [17]:
model_nb = GaussianNB(var_smoothing= 1.0)

model=model_nb
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.4587 (0.0435)
f1_macro: 0.4182 (0.0480)
precision_macro: 0.4615 (0.0638)
recall_macro: 0.4649 (0.0429)
kappa_scorer: 0.1936 (0.0640)


# Ann Shallow

In [46]:
# Tuning hyperparameters of shallow ANNs using grid search algorithm
model = MLPClassifier(max_iter=1000,  random_state=1)
hidden_layer_sizes= [(7),(8),(9),(10)]
activation= ['tanh', 'relu','identity','logistic']
solver= ['sgd', 'adam','lbfgs']
alpha= [0.0001, 0.1, 0.5, 1, 0.7]
learning_rate= ['constant','adaptive','invscaling']
grid= dict(
hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    alpha=alpha,
    learning_rate=learning_rate)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.799191 using {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': 10, 'learning_rate': 'constant', 'solver': 'lbfgs'}


C:\Users\Sep\anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [18]:
from sklearn.neural_network import MLPClassifier

model_ann = MLPClassifier(max_iter=1000,random_state=1, activation= 'logistic', alpha= 0.0001, hidden_layer_sizes= (10), learning_rate= 'constant', solver= 'lbfgs' )

model=model_ann
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.7747 (0.0548)
f1_macro: 0.7692 (0.0573)
precision_macro: 0.7804 (0.0560)
recall_macro: 0.7752 (0.0544)
kappa_scorer: 0.6621 (0.0819)


# Layer 2

In [47]:
# Finding the best architecture for proposed Deep-MLP-NN and Tuning its hyperparameters using grid search algorithm
layer_2= []
for a in range(7,10):
    for b in range(7,10):
        layer_2.append((a,b))
#2layer
model = MLPClassifier(max_iter=2000,  random_state=1)
hidden_layer_sizes= layer_2
activation= ['tanh', 'relu']
solver= ['sgd', 'adam','lbfgs']
alpha= [0.1, 1, 0.7]
learning_rate= ['constant','adaptive','invscaling']
grid= dict(
hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    alpha=alpha,
    learning_rate=learning_rate)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.853434 using {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (9, 7), 'learning_rate': 'constant', 'solver': 'lbfgs'}


C:\Users\Sep\anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [19]:
model_mlp_2layer = MLPClassifier(max_iter=2000,random_state=1, activation= 'tanh', alpha= 0.1, hidden_layer_sizes= (9, 7), learning_rate= 'constant', solver= 'lbfgs' )

model=model_mlp_2layer
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.8441 (0.0388)
f1_macro: 0.8363 (0.0428)
precision_macro: 0.8543 (0.0413)
recall_macro: 0.8442 (0.0386)
kappa_scorer: 0.7660 (0.0583)


# Layer 3

In [50]:
layer_3= []
for a in range(7,10):
    for b in range(7,10):
        for c in range(7,10):
            layer_3.append((a,b,c))
#3layer
model = MLPClassifier(max_iter=2000,  random_state=1)
hidden_layer_sizes= layer_3
activation= ['tanh', 'relu']
solver= ['adam','lbfgs']
alpha= [0.1, 0.7]
learning_rate= ['constant','adaptive']
grid= dict(
hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    alpha=alpha,
    learning_rate=learning_rate)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.864859 using {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (8, 8, 9), 'learning_rate': 'constant', 'solver': 'lbfgs'}


C:\Users\Sep\anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [20]:
model_mlp_3layer = MLPClassifier(max_iter=2000,random_state=1, activation= 'tanh', alpha= 0.1, hidden_layer_sizes= (8, 8, 9), learning_rate= 'constant', solver= 'lbfgs' )

model=model_mlp_3layer
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.8461 (0.0368)
f1_macro: 0.8391 (0.0394)
precision_macro: 0.8595 (0.0310)
recall_macro: 0.8460 (0.0369)
kappa_scorer: 0.7689 (0.0553)


# Layer 4

In [52]:
layer_4= []
for a in range(7,10):
    for b in range(7,10):
        for c in range(7,10):
            for d in range(7,10):
                layer_4.append((a,b,c,d))
#4layer
model = MLPClassifier(max_iter=2000,  random_state=1)
hidden_layer_sizes= layer_4
activation= ['tanh', 'relu']
solver= ['adam','lbfgs']
alpha= [0.1, 0.7]
learning_rate= ['constant','adaptive']
grid= dict(
hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    alpha=alpha,
    learning_rate=learning_rate)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.870767 using {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (7, 9, 8, 9), 'learning_rate': 'constant', 'solver': 'lbfgs'}


C:\Users\Sep\anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [21]:
model_mlp_4layer = MLPClassifier(max_iter = 2000,
                                 random_state = 1, 
                                 activation = 'tanh', 
                                 alpha = 0.1, 
                                 hidden_layer_sizes = (7, 9, 8, 9), 
                                 learning_rate = 'constant', 
                                 solver = 'lbfgs'
                                )

model=model_mlp_4layer
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.8597 (0.0351)
f1_macro: 0.8527 (0.0379)
precision_macro: 0.8767 (0.0315)
recall_macro: 0.8603 (0.0342)
kappa_scorer: 0.7895 (0.0526)


# Layer 5

In [55]:
layer_5= []
for a in range(7,10):
    for b in range(7,10):
        for c in range(7,10):
            for d in range(7,10):
                 for e in range(7,10):
                        layer_5.append((a,b,c,d,e))
#5layer
model = MLPClassifier(max_iter=2000,  random_state=1)
hidden_layer_sizes= layer_5
activation= ['tanh']
solver= ['adam','lbfgs']
alpha= [0.1]
learning_rate= ['constant']
grid= dict(
hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    alpha=alpha,
    learning_rate=learning_rate)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.867297 using {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (9, 8, 9, 9, 7), 'learning_rate': 'constant', 'solver': 'lbfgs'}


C:\Users\Sep\anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [22]:
model_mlp_5layer = MLPClassifier(max_iter=2000,
                                 random_state=1, 
                                 activation= 'tanh', 
                                 alpha= 0.1, 
                                 hidden_layer_sizes= (9, 8, 9, 9, 7), 
                                 learning_rate= 'constant', 
                                 solver= 'lbfgs' 
                                )

model=model_mlp_5layer
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.8284 (0.0457)
f1_macro: 0.8196 (0.0498)
precision_macro: 0.8384 (0.0467)
recall_macro: 0.8285 (0.0455)
kappa_scorer: 0.7424 (0.0687)


# Layer 6

In [57]:
layer_6= []
for a in range(7,10):
    for b in range(7,10):
        for c in range(7,10):
            for d in range(7,10):
                 for e in range(7,10):
                        for f in range(7,10):
                            layer_6.append((a,b,c,d,e,f))
#6layer
model = MLPClassifier(max_iter=2000,  random_state=1)
hidden_layer_sizes= layer_6
activation= ['tanh']
solver= ['lbfgs']
alpha= [0.1]
learning_rate= ['constant']
grid= dict(
hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    alpha=alpha,
    learning_rate=learning_rate)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
                           cv=cv, scoring='f1_macro',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.868773 using {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (8, 7, 7, 8, 8, 9), 'learning_rate': 'constant', 'solver': 'lbfgs'}


C:\Users\Sep\anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [23]:
model_mlp_6layer = MLPClassifier(max_iter=2000,
                                 random_state=1, 
                                 activation= 'tanh', 
                                 alpha= 0.1, 
                                 hidden_layer_sizes= (8, 7, 7, 8, 8, 9), 
                                 learning_rate= 'constant', 
                                 solver= 'lbfgs' 
                                )

model=model_mlp_6layer
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.7985 (0.0425)
f1_macro: 0.7910 (0.0453)
precision_macro: 0.8034 (0.0418)
recall_macro: 0.7991 (0.0415)
kappa_scorer: 0.6977 (0.0635)


# Stacking

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [25]:
# Select models with higher accuracy.
base_models = [
    #("svm", model_svm),
    ("knn", model_knn),
    ("rf", model_rf),
    #("dt", model_dt),
    #("nb", model_nb),
    #("ann", model_ann),
    #("mlp_2layer", model_mlp_2layer),
    ("mlp_3layer", model_mlp_3layer),
    ("mlp_4layer", model_mlp_4layer),
    #("mlp_5layer", model_mlp_5layer),
    #("mlp_6layer", model_mlp_6layer),
]

# Define your meta-learner (you can choose any classifier)
meta_learner = RandomForestClassifier(random_state=1)

# Create the stacking ensemble
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_learner)
model = stacking_model
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.9165 (0.0391)
f1_macro: 0.9167 (0.0388)
precision_macro: 0.9221 (0.0353)
recall_macro: 0.9165 (0.0396)
kappa_scorer: 0.8746 (0.0587)


In [27]:
# Select models with higher accuracy.
base_models = [
    #("svm", model_svm),
    ("knn", model_knn),
    ("rf", model_rf),
    #("dt", model_dt),
    #("nb", model_nb),
    #("ann", model_ann),
    #("mlp_2layer", model_mlp_2layer),
    ("mlp_3layer", model_mlp_3layer),
    ("mlp_4layer", model_mlp_4layer),
    #("mlp_5layer", model_mlp_5layer),
    #("mlp_6layer", model_mlp_6layer),
]

# Define your meta-learner (you can choose any classifier)
meta_learner = xgb.XGBClassifier(random_state=1)

# Create the stacking ensemble
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_learner)
model = stacking_model
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
print('f1_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='precision_macro', cv=cv, n_jobs=-1)
print('precision_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring='recall_macro', cv=cv, n_jobs=-1)
print('recall_macro: %.4f (%.4f)' % (mean(scores), std(scores)))
scores = cross_val_score(model, X, y, scoring=kappa_scorer, cv=cv, n_jobs=-1)
print('kappa_scorer: %.4f (%.4f)' % (mean(scores), std(scores)))

Accuracy: 0.9094 (0.0341)
f1_macro: 0.9090 (0.0338)
precision_macro: 0.9149 (0.0304)
recall_macro: 0.9092 (0.0346)
kappa_scorer: 0.8640 (0.0512)
